<a href="https://colab.research.google.com/github/akadavra/Module-5./blob/main/Hands_on_Lab_Build_an_Interactive_Dashboard_with_Ploty_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash==2.10.0 jupyter-dash==0.4.2 pandas plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [2]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

# Leer los datos
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

# Inicializar la app
app = JupyterDash(__name__)

# Layout de la app
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()] + [{'label': 'ALL', 'value': 'ALL'}],
        value='ALL',
        placeholder='Select a launch site',
        searchable=True
    ),

    html.Br(),

    html.Div(dcc.Graph(id='success-pie-chart')),

    html.Br(),

    html.P("Payload range (Kg):"),

    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=spacex_df['Payload Mass (kg)'].max(),
        step=1000,
        value=[0, spacex_df['Payload Mass (kg)'].max()]
    ),

    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback para la gráfica de pastel
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value'))
def update_pie_chart(site_dropdown):
    if site_dropdown == 'ALL':
        fig = px.pie(spacex_df, names='Launch Site', title='Total Successful Launches by Site')
    else:
        df_filtered = spacex_df[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df_filtered, names='class', title=f'Total Success vs Failure for site {site_dropdown}')
    return fig

# Callback para la gráfica de dispersión
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')])
def update_scatter_chart(site_dropdown, payload_range):
    low, high = payload_range
    df_filtered = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]

    if site_dropdown != 'ALL':
        df_filtered = df_filtered[df_filtered['Launch Site'] == site_dropdown]

    fig = px.scatter(
        df_filtered, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Correlation between Payload and Success'
    )
    return fig

# Mostrar la app en Google Colab
app.run_server(mode='inline')


Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>